<a href="https://colab.research.google.com/github/rubenesticesi/aprendizajeautomatico1/blob/master/PrediccionAnalisisCa%C3%B1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Análisis Exploratorio de Datos (EDA) para Producción de Caña de Azúcar
A continuación, presento el código Python para realizar un análisis exploratorio de datos (EDA) basado en los archivos proporcionados, enfocado en las variables TCH (Toneladas de Caña por Hectárea) y %Sac.Caña (Porcentaje de Sacarosa).

según los archivos anexos,  realizar un análisis  exploratorio de datos EDA, para identificar variables relevantes , detectar valores faltantes y posibles outliers, luego visualizar las variables de interés TCH - Toneladas de Caña por Hectárea Y %Sac.Caña - Porcentaje de Sacarosa.
A partir de estas 2 variables continuas, crear categorías que permitan clasificar los registros en niveles de desempeño.
Para %Sac.Caña, los niveles son Alto, Medio y Bajo.
Para TCH, los niveles son Alto, Medio y Bajo.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn-whitegrid')
sns.set_palette("pastel")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# ======================
# CARGA DE LOS DATASETS
# ======================

# Cargar los datasets
try:
    # Intentar cargar HISTORICO_SUERTES.xlsx que contiene las variables objetivo
    df_historico = pd.read_excel('HISTORICO_SUERTES.xlsx')

    # Intentar cargar BD_IPSA_1940.xlsx para el análisis de clasificación
    df_ipsa = pd.read_excel('BD_IPSA_1940.xlsx')

    print("Archivos cargados correctamente.")

except FileNotFoundError:
    print("Error: No se encontraron los archivos. Cargando datos de ejemplo basados en el fragmento proporcionado...")

    # Crear datos de ejemplo basados en el fragmento proporcionado
    # Para HISTORICO_SUERTES (simulando con los nombres de columnas identificados)
    columnas_historico = [
        'Período', 'Hacienda', 'Nombre', 'Zona', 'Tenencia', 'Suerte', 'Suelo', 'Area Neta', 'Dist Km', 'Variedad',
        'Cod.Estado #', 'Cod.Estado', 'F.Siembra', 'D.S.', 'Ult.Riego', 'Edad Ult Cos', 'F.Ult.Corte', 'Destino 1=Semilla',
        'Cod. T.Cultivo', 'Cultivo', 'Fec.Madur.', 'Producto', 'Dosis Madurante', 'Semanas mad.', 'TonUltCorte', 'TCH', 'TCHM',
        'Ton.Azucar', 'Rdto', 'TAH', 'TAHM', 'Sac.Caña Precosecha', 'Edad.Precosecha', '%Sac.Caña', '%Sac.Muestreadora',
        '%ATR', 'KATRHM', '%Fibra Caña', '%AR Jugo', '%ME Min', '%ME Veg', '%ME Tot', 'Brix', 'Pureza', 'Vejez', 'Tipo Quema',
        'T.Corte', 'Cerca de', 'Cosechó', 'Num.Riegos', 'M3 Riego', 'DDUlt.Riego', 'Lluvias (2 Meses Ant.)', 'Lluvias Ciclo',
        'Lluvias 0 -3', 'Lluvias tres a seis', 'Lluvias seis a nueve', 'Luvias 9'
    ]

    # Generar datos sintéticos para TCH y %Sac.Caña
    np.random.seed(42)
    n_samples = 1000

    # Simular datos realistas para TCH (toneladas por hectárea)
    tch_values = np.random.normal(loc=80, scale=20, size=n_samples)
    tch_values = np.clip(tch_values, 20, 150)  # Valores realistas entre 20 y 150 TCH

    # Simular datos realistas para %Sac.Caña
    sacarosa_values = np.random.normal(loc=13, scale=2, size=n_samples)
    sacarosa_values = np.clip(sacarosa_values, 8, 18)  # Valores realistas entre 8% y 18%

    # Crear DataFrame con datos sintéticos
    df_historico = pd.DataFrame({
        'TCH': tch_values,
        '%Sac.Caña': sacarosa_values,
        'Lluvias Ciclo': np.random.normal(loc=1000, scale=300, size=n_samples),
        'Edad.Precosecha': np.random.normal(loc=12, scale=2, size=n_samples),
        'Variedad': np.random.choice(['CC01-1940', 'SP79-1011', 'RB867515'], size=n_samples),
        'Tenencia': np.random.choice(['Propia', 'Arrendada', 'Sociedad'], size=n_samples),
        'Vejez': np.random.normal(loc=5, scale=2, size=n_samples)
    })

    # Para BD_IPSA_1940
    data_ipsa = [
        [1617, 'KATANOBU', 80573, '002A', 'Mecanizado Verde', 'CC01-1940', 'SI', 'BONUS 250 EC REGULADOR FISIOLÓGICO', 0.8, 10.3, 13.3, 1, 10.3, 0.4, 12.3, 6, '201906', 164, 0, 3, 2.6],
        [1618, 'KATANOBU', 80573, '002A', 'Mecanizado Verde', 'CC01-1940', 'SI', 'BONUS 250 EC REGULADOR FISIOLÓGICO', 0.8, 6.4, 11.8, 2, 8.3, 2, 12.9, 6, '202006', 130, 112, 3, 3.9],
        [1620, 'LA ALDEA ADIELA', 80270, '1', 'Mecanizado Verde', 'CC01-1940', 'SI', 'BONUS 250 EC REGULADOR FISIOLÓGICO', 1.2, 9.6, 12.4, 4, 7.6, 5.7, 14.6, 11, '201911', 123, 81, 2, 3.5]
    ]

    columnas_ipsa = ['ID', 'NOME', 'FAZ', 'TAL', 'tipocorte', 'variedad', 'madurada', 'producto', 'dosismad',
                    'semsmad', 'edad', 'cortes', 'me', 'vejez', 'sacarosa', 'mes', 'periodo', 'TCH',
                    'lluvias', 'grupo_tenencia', 'pct_diatrea']

    df_ipsa = pd.DataFrame(data_ipsa, columns=columnas_ipsa)

# ======================
# EDA PARA HISTORICO_SUERTES
# ======================

print("\n" + "="*50)
print("ANÁLISIS EXPLORATORIO DE DATOS - HISTORICO_SUERTES")
print("="*50)

# Información básica del dataset
print("\nInformación básica del dataset:")
print(f"Forma del dataset: {df_historico.shape}")
print(f"Número de columnas: {df_historico.shape[1]}")
print(f"Número de filas: {df_historico.shape[0]}")

# Mostrar las primeras filas
print("\nPrimeras 3 filas del dataset:")
print(df_historico.head(3))

# Verificar tipos de datos
print("\nTipos de datos de las columnas:")
print(df_historico.dtypes.value_counts())

# Identificar columnas relevantes para el análisis
print("\nColumnas relevantes identificadas para el análisis:")
relevant_columns = [col for col in df_historico.columns if 'TCH' in col or 'Sac' in col or 'sac' in col or 'lluvia' in col or 'edad' in col or 'vejez' in col or 'variedad' in col]
print(relevant_columns)

# ======================
# DETECCIÓN DE VALORES FALTANTES
# ======================

print("\n" + "="*50)
print("DETECCIÓN DE VALORES FALTANTES")
print("="*50)

# Calcular porcentaje de valores faltantes por columna
missing_values = df_historico.isnull().sum()
missing_percent = (missing_values / len(df_historico)) * 100

# Crear DataFrame con resultados
missing_data = pd.DataFrame({
    'Valores Faltantes': missing_values,
    '% Faltante': missing_percent
})

# Filtrar solo columnas con valores faltantes
missing_data = missing_data[missing_data['Valores Faltantes'] > 0].sort_values('% Faltante', ascending=False)

if not missing_data.empty:
    print("\nColumnas con valores faltantes:")
    print(missing_data.head(10))

    # Visualizar valores faltantes
    plt.figure(figsize=(14, 6))
    sns.heatmap(df_historico[relevant_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
    plt.title('Mapa de Calor de Valores Faltantes en Columnas Relevantes')
    plt.tight_layout()
    plt.savefig('valores_faltantes.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("✓ Mapa de calor de valores faltantes generado: 'valores_faltantes.png'")
else:
    print("\nNo se encontraron valores faltantes en el dataset.")

# ======================
# DETECCIÓN DE OUTLIERS
# ======================

print("\n" + "="*50)
print("DETECCIÓN DE OUTLIERS")
print("="*50)

# Identificar variables numéricas para análisis de outliers
numeric_cols = df_historico.select_dtypes(include=[np.number]).columns.tolist()

# Función para detectar outliers usando método de IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Buscar variables relevantes para TCH y %Sac.Caña
tch_col = next((col for col in df_historico.columns if 'TCH' in col), None)
sacarosa_col = next((col for col in df_historico.columns if 'Sac' in col or 'sac' in col), None)

if tch_col and sacarosa_col:
    print(f"\nVariables identificadas para análisis:")
    print(f"- TCH: '{tch_col}'")
    print(f"- %Sac.Caña: '{sacarosa_col}'")

    # Análisis de outliers para TCH
    tch_outliers, tch_lower, tch_upper = detect_outliers_iqr(df_historico, tch_col)
    print(f"\nAnálisis de outliers para {tch_col}:")
    print(f"- Rango normal: [{tch_lower:.2f}, {tch_upper:.2f}]")
    print(f"- Número de outliers: {len(tch_outliers)} ({len(tch_outliers)/len(df_historico)*100:.2f}%)")

    # Análisis de outliers para %Sac.Caña
    sac_outliers, sac_lower, sac_upper = detect_outliers_iqr(df_historico, sacarosa_col)
    print(f"\nAnálisis de outliers para {sacarosa_col}:")
    print(f"- Rango normal: [{sac_lower:.2f}, {sac_upper:.2f}]")
    print(f"- Número de outliers: {len(sac_outliers)} ({len(sac_outliers)/len(df_historico)*100:.2f}%)")

    # Visualización de outliers
    plt.figure(figsize=(14, 10))

    # Boxplot para TCH
    plt.subplot(2, 2, 1)
    sns.boxplot(y=df_historico[tch_col])
    plt.title(f'Boxplot de {tch_col}')
    plt.ylabel(tch_col)

    # Histograma para TCH
    plt.subplot(2, 2, 2)
    sns.histplot(df_historico[tch_col], kde=True)
    plt.axvline(x=tch_lower, color='r', linestyle='--', label=f'Lower Bound: {tch_lower:.2f}')
    plt.axvline(x=tch_upper, color='r', linestyle='--', label=f'Upper Bound: {tch_upper:.2f}')
    plt.title(f'Distribución de {tch_col}')
    plt.legend()

    # Boxplot para %Sac.Caña
    plt.subplot(2, 2, 3)
    sns.boxplot(y=df_historico[sacarosa_col])
    plt.title(f'Boxplot de {sacarosa_col}')
    plt.ylabel(sacarosa_col)

    # Histograma para %Sac.Caña
    plt.subplot(2, 2, 4)
    sns.histplot(df_historico[sacarosa_col], kde=True)
    plt.axvline(x=sac_lower, color='r', linestyle='--', label=f'Lower Bound: {sac_lower:.2f}')
    plt.axvline(x=sac_upper, color='r', linestyle='--', label=f'Upper Bound: {sac_upper:.2f}')
    plt.title(f'Distribución de {sacarosa_col}')
    plt.legend()

    plt.tight_layout()
    plt.savefig('distribucion_tch_sacarosa.png', dpi=300, bbox_inches='tight')
    print("✓ Visualización de distribución de TCH y %Sac.Caña generada: 'distribucion_tch_sacarosa.png'")

    # Análisis de correlación
    plt.figure(figsize=(12, 10))
    corr_matrix = df_historico[[tch_col, sacarosa_col] + [col for col in numeric_cols if col not in [tch_col, sacarosa_col]][:8]].corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Matriz de Correlación entre Variables Relevantes')
    plt.tight_layout()
    plt.savefig('correlacion_variables.png', dpi=300, bbox_inches='tight')
    print("✓ Matriz de correlación generada: 'correlacion_variables.png'")
else:
    print("\nNo se pudieron identificar las variables TCH y %Sac.Caña en el dataset.")

# ======================
# CREACIÓN DE CATEGORÍAS
# ======================

print("\n" + "="*50)
print("CREACIÓN DE CATEGORÍAS PARA TCH Y %SAC.CAÑA")
print("="*50)

if tch_col and sacarosa_col:
    # Para %Sac.Caña: Niveles Alto, Medio y Bajo
    # Usaremos percentiles 33 y 66 para dividir en tres categorías
    sacarosa_low = df_historico[sacarosa_col].quantile(0.33)
    sacarosa_high = df_historico[sacarosa_col].quantile(0.66)

    print(f"\nUmbrales para %Sac.Caña:")
    print(f"- Bajo: < {sacarosa_low:.2f}%")
    print(f"- Medio: {sacarosa_low:.2f}% - {sacarosa_high:.2f}%")
    print(f"- Alto: > {sacarosa_high:.2f}%")

    # Crear columna de categoría para %Sac.Caña
    df_historico['Categoria_Sacarosa'] = pd.cut(
        df_historico[sacarosa_col],
        bins=[-np.inf, sacarosa_low, sacarosa_high, np.inf],
        labels=['Bajo', 'Medio', 'Alto']
    )

    # Para TCH: Niveles Alto, Medio y Bajo
    tch_low = df_historico[tch_col].quantile(0.33)
    tch_high = df_historico[tch_col].quantile(0.66)

    print(f"\nUmbrales para TCH:")
    print(f"- Bajo: < {tch_low:.2f} TCH")
    print(f"- Medio: {tch_low:.2f} - {tch_high:.2f} TCH")
    print(f"- Alto: > {tch_high:.2f} TCH")

    # Crear columna de categoría para TCH
    df_historico['Categoria_TCH'] = pd.cut(
        df_historico[tch_col],
        bins=[-np.inf, tch_low, tch_high, np.inf],
        labels=['Bajo', 'Medio', 'Alto']
    )

    # Visualizar distribución de categorías
    plt.figure(figsize=(14, 6))

    # Distribución de categorías de Sacarosa
    plt.subplot(1, 2, 1)
    sns.countplot(x='Categoria_Sacarosa', data=df_historico, order=['Bajo', 'Medio', 'Alto'])
    plt.title('Distribución de Categorías de %Sac.Caña')
    plt.xlabel('%Sac.Caña')
    plt.ylabel('Frecuencia')

    # Añadir porcentajes en las barras
    for p in plt.gca().patches:
        height = p.get_height()
        plt.gca().annotate(f'{height/len(df_historico)*100:.1f}%',
                           (p.get_x() + p.get_width()/2., height),
                           ha='center', va='center',
                           xytext=(0, 5),
                           textcoords='offset points')

    # Distribución de categorías de TCH
    plt.subplot(1, 2, 2)
    sns.countplot(x='Categoria_TCH', data=df_historico, order=['Bajo', 'Medio', 'Alto'])
    plt.title('Distribución de Categorías de TCH')
    plt.xlabel('TCH')
    plt.ylabel('Frecuencia')

    # Añadir porcentajes en las barras
    for p in plt.gca().patches:
        height = p.get_height()
        plt.gca().annotate(f'{height/len(df_historico)*100:.1f}%',
                           (p.get_x() + p.get_width()/2., height),
                           ha='center', va='center',
                           xytext=(0, 5),
                           textcoords='offset points')

    plt.tight_layout()
    plt.savefig('distribucion_categorias.png', dpi=300, bbox_inches='tight')
    print("✓ Distribución de categorías generada: 'distribucion_categorias.png'")

    # Análisis cruzado de categorías
    plt.figure(figsize=(10, 8))
    cross_tab = pd.crosstab(df_historico['Categoria_Sacarosa'], df_historico['Categoria_TCH'])
    sns.heatmap(cross_tab, annot=True, fmt='d', cmap='YlGnBu', linewidths=.5)
    plt.title('Relación entre Categorías de %Sac.Caña y TCH')
    plt.ylabel('%Sac.Caña')
    plt.xlabel('TCH')
    plt.tight_layout()
    plt.savefig('relacion_categorias.png', dpi=300, bbox_inches='tight')
    print("✓ Relación entre categorías generada: 'relacion_categorias.png'")

    # Mostrar proporciones
    print("\nProporción de registros en cada categoría:")
    print(f"- %Sac.Caña:")
    print(df_historico['Categoria_Sacarosa'].value_counts(normalize=True).sort_index() * 100)
    print(f"\n- TCH:")
    print(df_historico['Categoria_TCH'].value_counts(normalize=True).sort_index() * 100)

    # Análisis por variedad (si está disponible)
    if 'Variedad' in df_historico.columns:
        plt.figure(figsize=(14, 10))

        # Sacarosa por variedad
        plt.subplot(2, 1, 1)
        sns.boxplot(x='Variedad', y=sacarosa_col, data=df_historico)
        plt.title(f'Distribución de %Sac.Caña por Variedad')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # TCH por variedad
        plt.subplot(2, 1, 2)
        sns.boxplot(x='Variedad', y=tch_col, data=df_historico)
        plt.title(f'Distribución de TCH por Variedad')
        plt.xticks(rotation=45)
        plt.tight_layout()

        plt.savefig('analisis_por_variedad.png', dpi=300, bbox_inches='tight')
        print("✓ Análisis por variedad generado: 'analisis_por_variedad.png'")

    # Guardar el DataFrame con las nuevas categorías
    df_historico.to_csv('HISTORICO_SUERTES_con_categorias.csv', index=False)
    print("\n✓ Dataset con categorías guardado como 'HISTORICO_SUERTES_con_categorias.csv'")

# ======================
# ANÁLISIS ADICIONAL CON BD_IPSA_1940
# ======================

print("\n" + "="*50)
print("ANÁLISIS ADICIONAL CON BD_IPSA_1940")
print("="*50)

# Verificar si las variables están presentes en BD_IPSA_1940
tch_col_ipsa = next((col for col in df_ipsa.columns if 'TCH' in col), None)
sacarosa_col_ipsa = next((col for col in df_ipsa.columns if 'sacarosa' in col or 'Sac' in col), None)

if tch_col_ipsa and sacarosa_col_ipsa:
    print(f"\nVariables identificadas en BD_IPSA_1940:")
    print(f"- TCH: '{tch_col_ipsa}'")
    print(f"- Sacarosa: '{sacarosa_col_ipsa}'")

    # Crear categorías para BD_IPSA_1940 usando los mismos umbrales
    df_ipsa['Categoria_Sacarosa'] = pd.cut(
        df_ipsa[sacarosa_col_ipsa],
        bins=[-np.inf, sacarosa_low, sacarosa_high, np.inf],
        labels=['Bajo', 'Medio', 'Alto']
    )

    df_ipsa['Categoria_TCH'] = pd.cut(
        df_ipsa[tch_col_ipsa],
        bins=[-np.inf, tch_low, tch_high, np.inf],
        labels=['Bajo', 'Medio', 'Alto']
    )

    print("\nDistribución de categorías en BD_IPSA_1940:")
    print(f"- %Sac.Caña: {df_ipsa['Categoria_Sacarosa'].value_counts().to_dict()}")
    print(f"- TCH: {df_ipsa['Categoria_TCH'].value_counts().to_dict()}")

    # Guardar el DataFrame con las nuevas categorías
    df_ipsa.to_csv('BD_IPSA_1940_con_categorias.csv', index=False)
    print("\n✓ Dataset con categorías guardado como 'BD_IPSA_1940_con_categorias.csv'")
else:
    print("\nNo se pudieron identificar las variables TCH y sacarosa en BD_IPSA_1940.")

print("\n" + "="*50)
print("ANÁLISIS COMPLETADO")
print("="*50)

**Explicación del Código**

Este código realiza un análisis exploratorio de datos (EDA) completo para los datasets proporcionados, con enfoque en las variables TCH (%Sac.Caña) y %Sac.Caña, siguiendo los requisitos del taller práctico:

1. Carga de Datos
Carga los archivos Excel HISTORICO_SUERTES.xlsx y BD_IPSA_1940.xlsx
Si los archivos no están disponibles, genera datos sintéticos basados en la estructura proporcionada en los fragmentos
2. Análisis Exploratorio Básico
Muestra información básica del dataset (forma, tipos de datos)
Identifica columnas relevantes para el análisis de TCH y %Sac.Caña
3. Detección de Valores Faltantes
Calcula el porcentaje de valores faltantes por columna
Genera un mapa de calor para visualizar la distribución de valores faltantes
4. Detección de Outliers
Utiliza el método del rango intercuartílico (IQR) para identificar outliers
Genera visualizaciones (boxplots e histogramas) para TCH y %Sac.Caña
Crea una matriz de correlación para identificar relaciones entre variables
5. Creación de Categorías
Define umbrales para categorizar %Sac.Caña en niveles Alto, Medio y Bajo usando percentiles 33 y 66
Define umbrales para categorizar TCH en niveles Alto, Medio y Bajo usando los mismos percentiles
Genera visualizaciones de la distribución de estas categorías
Analiza la relación entre las categorías de TCH y %Sac.Caña
Si está disponible, realiza un análisis por variedad de caña
6. Análisis Adicional con BD_IPSA_1940

Aplica los mismos umbrales de categorización al dataset BD_IPSA_1940
Muestra la distribución de categorías en este dataset

El código genera múltiples visualizaciones y guarda los datasets con las nuevas categorías para su uso posterior en modelos de clasificación, tal como se requiere en el taller práctico.

Este análisis proporciona una base sólida para entender la distribución de las variables clave, identificar posibles problemas en los datos y establecer criterios para la categorización que será utilizada en los modelos de clasificación.

Otra Forma Sugerida
El siguiente análisis exploratorio de datos (EDA) en Python identifica variables relevantes, valores faltantes y outliers, y visualiza las variables clave TCH y %Sac.Caña. Además, el código crea categorías de desempeño (Alto, Medio, Bajo) para TCH y %Sac.Caña usando percentiles como criterio.

Carga y preparación de los datos

In [ ]:
# Carga y preparación de los datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar los archivos Excel
df = pd.read_excel('HISTORICO_SUERTES.xlsx')
df_ipsa = pd.read_excel('BD_IPSA_1940.xlsx')

# Seleccionar variables de interés
variables_interes = ['TCH', '%Sac.Caña']  # TCH: Toneladas de Caña por Hectárea, %Sac.Caña: Porcentaje de Sacarosa
eda_df = df[variables_interes].copy()


Análisis de valores faltantes y outliers

In [ ]:
# Valores faltantes
print('Valores faltantes:\n', eda_df.isnull().sum())

# Resumen estadístico
print('Resumen estadístico:\n', eda_df.describe())

# Detección de Outliers con boxplot
plt.figure(figsize=(12, 5))
for idx, col in enumerate(variables_interes):
    plt.subplot(1, 2, idx+1)
    sns.boxplot(y=eda_df[col], color='skyblue')
    plt.title(f'Boxplot {col}')
plt.tight_layout()
plt.show()


Visualización de la distribución

In [ ]:
plt.figure(figsize=(12, 5))
for idx, col in enumerate(variables_interes):
    plt.subplot(1, 2, idx+1)
    sns.histplot(eda_df[col].dropna(), kde=True, bins=30, color='green')
    plt.title(f'Distribución {col}')
plt.xlabel(col)
plt.ylabel('Frecuencia')
plt.grid(True)
plt.tight_layout()
plt.show()


Creación de categorías por percentiles

In [ ]:
# Definir percentiles para los cortes
for col in variables_interes:
    bajos = eda_df[col].quantile(0.33)
    medios = eda_df[col].quantile(0.66)
    print(f'{col}: Bajo <= {bajos:.2f}, Medio <= {medios:.2f}, Alto > {medios:.2f}')

# Función para categorizar
def categorizar(valor, bajos, medios):
    if valor <= bajos:
        return 'Bajo'
    elif valor <= medios:
        return 'Medio'
    else:
        return 'Alto'

for col in variables_interes:
    bajos = eda_df[col].quantile(0.33)
    medios = eda_df[col].quantile(0.66)
    eda_df[f'{col}_Nivel'] = eda_df[col].apply(lambda x: categorizar(x, bajos, medios))

print(eda_df[[*variables_interes, f'{variables_interes}_Nivel', f'{variables_interes[1]}_Nivel']].head())



Este código debe adaptarse si el nombre de columnas en los archivos varía (puede ser 'Sac.Caña', '%Sac.Caña', etc.), valide con df.columns.

La creación de categorías usa percentiles (33, 66), apropiado para segmentación comparativa cuando no hay criterio de negocio explícito.

Las visualizaciones permiten identificar la presencia visual de outliers y la forma de las distribuciones.

El EDA puede ampliarse integrando otras variables pertinentes o cruzando con el segundo archivo para análisis más profundo según el contexto.



Commencing Data Exploration

I've begun my initial EDA by loading the provided CSV files into pandas dataframes. Next, I'll examine the structure and content, noting data types and looking for any immediate data cleaning needs. I'll be sure to provide you a comprehensive initial overview.

Beginning the Analysis

I've loaded the data and performed the initial exploration steps, noting data types, null counts, and descriptive statistics. It appears HISTORICO_SUERTES.xlsx is the main file for the regression analysis. I'm focusing on TCH and %Sac.Caña, and I've identified sacarosa in BD_IPSA_1940.xlsx as a potentially equivalent variable. I'm calculating missing value percentages now.

Delving Into the Data

I've loaded the data, and the initial exploration is underway. The HISTORICO_SUERTES.xlsx file reveals the presence of both TCH and %Sac.Caña, confirming its suitability for the core regression analysis. I'm focusing on their distributions and potential outliers using boxplots and statistical methods. Additionally, I'm examining BD_IPSA_1940.xlsx to confirm sacarosa corresponds to %Sac.Caña. Finally, I am creating categories of TCH and %Sac.Caña into "Alto," "Medio," and "Bajo" groups using quantiles.

Expanding the Analysis

I've loaded both datasets and performed the initial exploration as planned. I've focused on data types, null counts, and descriptive statistics. Now, I'm carefully examining HISTORICO_SUERTES.xlsx to understand the variables present, distributions, and outliers for TCH and %Sac.Caña. I'm starting the categorization into 'Alto', 'Medio', and 'Bajo' groups using the quantile method and preparing visual representations. I'm also preparing to analyze BD_IPSA_1940.xlsx to establish the relationship between variables, as planned.

Defining the Approach

I've completed loading and the initial exploration of both datasets. I'm focusing on handling missing values and outlier detection, employing boxplots and quantile methods for TCH and %Sac.Caña. I'm verifying the correspondence of sacarosa from BD_IPSA_1940.xlsx with %Sac.Caña, as the datasets may need to be integrated for the ultimate task. My next step will be to create categories and visualize the relationships.

Claro, aquí tienes el análisis exploratorio de datos (EDA) y la categorización de las variables TCH y %Sac.Caña a partir de los archivos que proporcionaste.

Análisis Exploratorio de Datos (EDA)
Realicé un análisis de los dos archivos de datos: HISTORICO_SUERTES.xlsx y BD_IPSA_1940.xlsx. El objetivo es entender la estructura de los datos, identificar variables clave, detectar valores faltantes y encontrar posibles valores atípicos (outliers).

Dataset 1: HISTORICO_SUERTES.xlsx
Este es el conjunto de datos principal para la tarea de regresión.

1. Variables Relevantes y Estructura
El dataset contiene 94 columnas y 4,858 registros. Las variables objetivo principales son:

TCH: Toneladas de Caña por Hectárea.

%Sac.Caña: Porcentaje de Sacarosa en la caña.

Otras variables potencialmente relevantes incluyen Edad Ult Cos (Edad del cultivo en la última cosecha), Num.Riegos, Lluvias Ciclo, y datos de fertilizantes y temperatura.

2. Valores Faltantes
Se encontraron valores faltantes en varias columnas. Las más críticas son:

%Sac.Caña: Tiene 1,332 valores faltantes (27.4% del total). Esto es muy importante, ya que es una de las variables a predecir.

Sac.Caña Precosecha: 1,323 valores faltantes (27.2%).

Columnas de fertilizantes y madurantes: Varias columnas como Dosis Madurante y Fosfato Jugo tienen un alto porcentaje de datos nulos, superando en algunos casos el 80%.

La presencia de tantos datos faltantes en la variable %Sac.Caña requerirá una estrategia cuidadosa, como la eliminación de los registros o el uso de técnicas de imputación antes de entrenar un modelo.

3. Análisis de Outliers y Visualización
TCH - Toneladas de Caña por Hectárea
Distribución: La distribución de TCH está sesgada a la derecha. La mayoría de las cosechas producen entre 100 y 150 toneladas por hectárea.

Outliers: Se observan valores atípicos significativos, con algunos registros superando las 300 TCH, lo cual es excepcionalmente alto y podría deberse a errores de registro o a condiciones de cultivo únicas.

Estadísticas de TCH:

Media: 136.9 TCH

Mediana: 133.7 TCH

Valor Máximo: 343.8 TCH

%Sac.Caña - Porcentaje de Sacarosa
Distribución: La sacarosa muestra una distribución más centrada y simétrica, con la mayoría de los valores concentrados entre 11% y 14%.

Outliers: Existen algunos valores atípicamente bajos (menores a 6%) y altos (mayores a 17%), que podrían ser considerados para revisión.

Estadísticas de %Sac.Caña:

Media: 12.5%

Mediana: 12.7%

Valor Máximo: 17.6%

Dataset 2: BD_IPSA_1940.xlsx
Este dataset parece enfocado en una variedad específica (CC01-1940) y se usará para la tarea de clasificación.

Estructura: Contiene 21 columnas y 1,655 registros. Las variables de interés son TCH y sacarosa.

Valores Faltantes: Este conjunto de datos está mucho más limpio, no presenta valores nulos en las columnas de TCH y sacarosa.

Visualización: Las distribuciones de TCH y sacarosa en este archivo son muy similares a las del primer dataset, validando los patrones observados.

Creación de Categorías de Desempeño
Para clasificar los registros en niveles de desempeño Bajo, Medio y Alto, se utilizaron los cuantiles. Este método divide los datos en grupos con un número igual de observaciones. Se usaron los terciles (cuantiles 0.33 y 0.66) para crear tres categorías balanceadas.

A continuación, se muestran los umbrales calculados a partir del archivo HISTORICO_SUERTES.xlsx:

Categorías para TCH
Los puntos de corte para clasificar las Toneladas de Caña por Hectárea son:

Bajo: Menor o igual a 118.8 TCH

Medio: Entre 118.8 TCH y 151.0 TCH

Alto: Mayor a 151.0 TCH

Distribución de las Categorías de TCH
El gráfico de barras muestra cómo se distribuyen los registros en estas tres nuevas categorías. Como se usaron cuantiles, la distribución es bastante equilibrada.

Categorías para %Sac.Caña
Los puntos de corte para clasificar el Porcentaje de Sacarosa son:

Bajo: Menor o igual a 11.8%

Medio: Entre 11.8% y 13.4%

Alto: Mayor a 13.4%

Distribución de las Categorías de %Sac.Caña
Al igual que con TCH, la distribución de las categorías de sacarosa es uniforme debido al método de cuantiles.

Estas nuevas columnas categóricas (Nivel_TCH y Nivel_Sacarosa) están listas para ser utilizadas como variables objetivo en un modelo de clasificación, permitiendo al ingenio identificar y analizar los factores que llevan a un desempeño bajo, medio o alto.









In [ ]:
# Paso 1: Importar las librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para un mejor estilo en los gráficos
sns.set(style="whitegrid")

# --- ANÁLISIS DEL DATASET: HISTORICO_SUERTES.xlsx ---

print("="*50)
print("Análisis del archivo: HISTORICO_SUERTES.xlsx")
print("="*50)

# Paso 2: Cargar el primer conjunto de datos
try:
    df_historico = pd.read_csv('HISTORICO_SUERTES.xlsx - Hoja1.csv')
    print("Archivo 'HISTORICO_SUERTES.xlsx - Hoja1.csv' cargado exitosamente.")
    print(f"El dataset tiene {df_historico.shape[0]} filas y {df_historico.shape[1]} columnas.\n")

    # Paso 3: Análisis exploratorio de datos (EDA)

    # Revisar las primeras filas
    print("--- Primeras 5 filas del dataset ---")
    print(df_historico.head())
    print("\n")

    # Obtener información general y tipos de datos
    print("--- Información general del dataset ---")
    df_historico.info()
    print("\n")

    # Identificar valores faltantes
    print("--- Porcentaje de valores faltantes (Top 10) ---")
    missing_values = df_historico.isnull().sum() / len(df_historico) * 100
    print(missing_values[missing_values > 0].sort_values(ascending=False).head(10))
    print("\n")

    # Estadísticas descriptivas de las variables de interés
    print("--- Estadísticas descriptivas para TCH y %Sac.Caña ---")
    print(df_historico[['TCH', '%Sac.Caña']].describe())
    print("\n")

    # Paso 4: Visualización de TCH y %Sac.Caña

    # Visualización para TCH
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(df_historico['TCH'], kde=True, bins=30)
    plt.title('Distribución de TCH (Toneladas de Caña/Hectárea)')
    plt.xlabel('TCH')
    plt.ylabel('Frecuencia')

    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_historico['TCH'])
    plt.title('Boxplot de TCH para detectar outliers')
    plt.xlabel('TCH')
    plt.tight_layout()
    plt.show()

    # Visualización para %Sac.Caña (eliminando valores nulos para el gráfico)
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(df_historico['%Sac.Caña'].dropna(), kde=True, bins=30, color='green')
    plt.title('Distribución de %Sac.Caña (Porcentaje de Sacarosa)')
    plt.xlabel('% Sacarosa')
    plt.ylabel('Frecuencia')

    plt.subplot(1, 2, 2)
    sns.boxplot(x=df_historico['%Sac.Caña'].dropna(), color='green')
    plt.title('Boxplot de %Sac.Caña para detectar outliers')
    plt.xlabel('% Sacarosa')
    plt.tight_layout()
    plt.show()

    # Paso 5: Creación de categorías de desempeño

    print("--- Creando categorías de desempeño para TCH y %Sac.Caña ---")

    # Definir etiquetas para las categorías
    labels = ['Bajo', 'Medio', 'Alto']

    # Crear categorías para TCH usando terciles (3 grupos de igual tamaño)
    df_historico['Nivel_TCH'] = pd.qcut(df_historico['TCH'], q=3, labels=labels)

    # Crear categorías para %Sac.Caña (manejando valores nulos)
    # Se categorizan solo los valores no nulos
    df_historico['Nivel_Sacarosa'] = pd.qcut(df_historico['%Sac.Caña'][df_historico['%Sac.Caña'].notna()], q=3, labels=labels)

    print("Categorías creadas exitosamente.\n")

    # Mostrar la distribución de las nuevas categorías
    print("--- Distribución de niveles de TCH ---")
    print(df_historico['Nivel_TCH'].value_counts())
    print("\n")
    print("--- Distribución de niveles de %Sac.Caña ---")
    print(df_historico['Nivel_Sacarosa'].value_counts())
    print("\n")

    # Visualizar la distribución de las categorías
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.countplot(x='Nivel_TCH', data=df_historico, palette='viridis', order=labels)
    plt.title('Distribución de Niveles de TCH')

    plt.subplot(1, 2, 2)
    sns.countplot(x='Nivel_Sacarosa', data=df_historico, palette='plasma', order=labels)
    plt.title('Distribución de Niveles de %Sac.Caña')
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("Error: No se encontró el archivo 'HISTORICO_SUERTES.xlsx - Hoja1.csv'")


# --- ANÁLISIS DEL DATASET: BD_IPSA_1940.xlsx ---

print("\n" + "="*50)
print("Análisis del archivo: BD_IPSA_1940.xlsx")
print("="*50)

# Paso 6: Cargar y analizar el segundo conjunto de datos
try:
    df_ipsa = pd.read_csv('BD_IPSA_1940.xlsx - BD_IPSA.csv')
    print("Archivo 'BD_IPSA_1940.xlsx - BD_IPSA.csv' cargado exitosamente.")
    print(f"El dataset tiene {df_ipsa.shape[0]} filas y {df_ipsa.shape[1]} columnas.\n")

    # Revisar si hay valores faltantes
    print("--- Revisión de valores faltantes en BD_IPSA ---")
    print(df_ipsa[['TCH', 'sacarosa']].isnull().sum())
    print("El dataset no tiene valores faltantes en las columnas de interés.\n")

    # Crear categorías también para este dataset para la tarea de clasificación
    df_ipsa['Nivel_TCH'] = pd.qcut(df_ipsa['TCH'], q=3, labels=labels)
    df_ipsa['Nivel_Sacarosa'] = pd.qcut(df_ipsa['sacarosa'], q=3, labels=labels)

    print("--- Distribución de niveles de TCH (BD_IPSA) ---")
    print(df_ipsa['Nivel_TCH'].value_counts())
    print("\n")
    print("--- Distribución de niveles de Sacarosa (BD_IPSA) ---")
    print(df_ipsa['Nivel_Sacarosa'].value_counts())
    print("\n")

except FileNotFoundError:
    print("Error: No se encontró el archivo 'BD_IPSA_1940.xlsx - BD_IPSA.csv'")